In [1]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib as mpl

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_meter, get_data, init_distributed_device
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp
from collections import OrderedDict

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def unwrap_model(state_dict):
    new_state_dict = OrderedDict()
    for key, value in state_dict.items():
        new_key = key.replace('module.', '')
        new_state_dict[new_key] = value
    
    return new_state_dict

def norm_image(img):
    img = img - img.min()
    img = img / img.max()
    return img

In [3]:
# exp_name = 'mae_cm_reconall_scratch_allTrain'
# exp_name = 'mae_unidecoder_scratch_allTrain_norwl'
# exp_name = 'sanity_check'
n_spectral_blocks = 1
n_spatial_spectral_blocks = 2
n_spatial_blocks = 9
exp_name = f"mm_mae_fcvit_{n_spectral_blocks}-{n_spatial_spectral_blocks}-{n_spatial_blocks}_bn"
# # exp_name = "mae_base_cvit"
# device = torch.device('cuda:2')
# device = torch.device('cpu')

In [4]:
args = {'exp_name': exp_name,
        'config_file': 'GeospatialFM/configs/pretrain_mm_cvit_bn.yaml',
        'opts': None, 
        'save_config': False,
        'device': 0}
args = argparse.Namespace(**args)
args.debug = True
args.finetune = False
device = init_distributed_device(args)
cfg, _ = setup(args)
cfg.MODEL.MULTI_MODAL.kwargs.spectral_blocks = n_spectral_blocks
cfg.MODEL.MULTI_MODAL.kwargs.sptial_spectral_blocks = n_spatial_spectral_blocks
cfg.DATASET.task_type = 'multilabel'
args

Namespace(exp_name='mm_mae_fcvit_1-2-9_bn', config_file='GeospatialFM/configs/pretrain_mm_cvit_bn.yaml', opts=None, save_config=False, device='cuda:0', debug=True, finetune=False, distributed=False, world_size=1, rank=0, local_rank=0)

In [5]:
save_path = os.path.join(cfg.TRAINER['output_dir'], 'final_model.pth')
# save_path = os.path.join(cfg.TRAINER['output_dir'], 'ckpt_epoch10.pth')

model = construct_mae(cfg.MODEL)
state_dict = unwrap_model(torch.load(save_path, map_location='cpu'))
model.load_state_dict(state_dict, strict=False)
model = model.to(device)

# print the number of parameters for model
num_params = sum(p.numel() for p in model.encoder.parameters())
print(f'Number of parameters: {num_params}')

Constructing mm_cvit_base_patch16_224 encoder...
Spectral Blocks: 1	Sptial-Spectral Blocks: 2	Spatial Blocks: 9
Number of parameters: 95456256


In [6]:
num_params = sum(p.numel() for p in model.encoder.radar_spectral_blocks.parameters())
print(f'Number of parameters: {num_params}')

Number of parameters: 7087872


In [7]:
data_idx = 1000 #41982
data = get_data(cfg)
sample = data['test'].dataloader.dataset[data_idx]
for key in sample:
    sample[key] = sample[key].unsqueeze(0)

Training Dataset: BigEarthNet
Train Set: 393418	 Val Set: 12586	 Test Set: 125866


In [ ]:
with torch.no_grad():
    optical_out = model.forward_recon(sample['image'].to(device), None, slice_patch_tokens=(13, 2), channel_mask_ratio=0., mask_ratio=0.75)
    radar_out = model.forward_recon(None, sample['radar'].to(device), slice_patch_tokens=(13, 2), channel_mask_ratio=0., mask_ratio=0.75)
    mm_out = model.forward_recon(sample['image'].to(device), sample['radar'].to(device), slice_patch_tokens=(13, 2), channel_mask_ratio=0., mask_ratio=0.75)

In [ ]:
optical_cls = optical_out['cls_token']
radar_cls = radar_out['cls_token']
mm_cls = mm_out['cls_token']

# calculate the cosine similarity
cos_sim = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
optical_radar_sim = cos_sim(optical_cls, radar_cls)
optical_mm_sim = cos_sim(optical_cls, mm_cls)
radar_mm_sim = cos_sim(radar_cls, mm_cls)

print(f'Optical Radar Similarity: {optical_radar_sim.item()}')
print(f'Optical MM Similarity: {optical_mm_sim.item()}')
print(f'Radar MM Similarity: {radar_mm_sim.item()}')

In [ ]:
sample_idx = 0
rgb_channel = [3, 2, 1]

optical_recon = optical_out['recon'].cpu().numpy()
radar_recon = radar_out['recon'].cpu().numpy()
mm_recon = mm_out['recon'].cpu().numpy()
optical_input = sample['image'].cpu().numpy()
radar_input = sample['radar'].cpu().numpy()

optical_mask = optical_out['mask'].cpu().numpy()
radar_mask = radar_out['mask'].cpu().numpy()
optical_channel_mask = optical_out.get('optical_channel_mask', None)
mm_channel_mask = mm_out.get('optical_channel_mask', None)
if optical_channel_mask is not None: 
    optical_channel_mask = optical_channel_mask.reshape(-1, 256, 13)[0, 0]
    optical_channel_mask = optical_channel_mask.cpu().numpy()
if mm_channel_mask is not None:
    mm_channel_mask = mm_channel_mask.reshape(-1, 256, 13)[0, 0]
    mm_channel_mask = mm_channel_mask.cpu().numpy()

vis_optical_mask = optical_mask[sample_idx].reshape(14, 14)
vis_radar_mask = radar_mask[sample_idx].reshape(14, 14)
# scale up the mask from 14x14 to 224x224 in chunks of 16x16
vis_optical_mask = np.repeat(np.repeat(vis_optical_mask, 16, axis=0), 16, axis=1)
vis_radar_mask = np.repeat(np.repeat(vis_radar_mask, 16, axis=0), 16, axis=1)
optical_recon_all = optical_recon[sample_idx].transpose(1, 2, 0)
radar_recon_all = radar_recon[sample_idx].transpose(1, 2, 0)
mm_recon_all = mm_recon[sample_idx].transpose(1, 2, 0)
optical_input = optical_input[sample_idx].transpose(1, 2, 0)
radar_input = radar_input[sample_idx].transpose(1, 2, 0)

if optical_recon_all.shape[2] >= 13:
    vis_optical_recon_o = optical_recon_all[:, :, rgb_channel]
    vis_optical_recon_o = norm_image(vis_optical_recon_o)
else:
    vis_optical_recon_o = None

if optical_recon_all.shape[2] > 13:
    vis_optical_recon_r = optical_recon_all[:, :, 13]
    # vis_optical_recon_r = norm_image(vis_optical_recon_r)
elif optical_recon_all.shape[2] == 13:
    vis_optical_recon_r = None
else:
    vis_optical_recon_r = optical_recon_all[:, :, 0]
    # vis_optical_recon_r = norm_image(vis_optical_recon_r)

if radar_recon_all.shape[2] >= 13:
    vis_radar_recon_o = radar_recon_all[:, :, rgb_channel]
    vis_radar_recon_o = norm_image(vis_radar_recon_o)
else:
    vis_radar_recon_o = None

if radar_recon_all.shape[2] > 13:
    vis_radar_recon_r = radar_recon_all[:, :, 13]
    vis_radar_recon_r = norm_image(vis_radar_recon_r)
elif radar_recon_all.shape[2] == 13:
    vis_radar_recon_r = None
else:
    vis_radar_recon_r = radar_recon_all[:, :, 0]
    # vis_radar_recon_r = norm_image(vis_radar_recon_r)

vis_optical_input = optical_input[:, :, rgb_channel]
vis_radar_input = radar_input[:, :, 0]
vis_optical_input = norm_image(vis_optical_input)
# vis_radar_input = norm_image(vis_radar_input)

if vis_optical_recon_o is not None:
    vis_optical_recon_o = vis_optical_recon_o * vis_optical_mask[:, :, None] + vis_optical_input * (1 - vis_optical_mask[:, :, None])
if vis_optical_recon_r is not None:
    vis_optical_recon_r = vis_optical_recon_r * vis_optical_mask + vis_radar_input * (1 - vis_optical_mask)
if vis_radar_recon_o is not None:
    vis_radar_recon_o = vis_radar_recon_o * vis_radar_mask[:, :, None] + vis_optical_input * (1 - vis_radar_mask[:, :, None])
if vis_radar_recon_r is not None:
    vis_radar_recon_r = vis_radar_recon_r * vis_radar_mask + vis_radar_input * (1 - vis_radar_mask)

# # normalize all images
# vis_optical_recon_o = norm_image(vis_optical_recon_o) if vis_optical_recon_o is not None else None
# vis_optical_input = norm_image(vis_optical_input)
# vis_radar_recon_o = norm_image(vis_radar_recon_o) if vis_radar_recon_o is not None else None


In [ ]:
optical_recon.std(), optical_recon.mean(), radar_recon.std(), radar_recon.mean()

In [ ]:
optical_input.std(), optical_input.mean(), radar_input.std(), radar_input.mean()

In [ ]:
# plot a graph with 2x3 subplots
fig, ax = plt.subplots(3, 4, figsize=(20, 15))
# set font size
plt.rcParams.update({'font.size': 16})

ax[0, 1].imshow(vis_optical_input)
ax[0, 1].set_title('Optical Input')
ax[0, 2].imshow(vis_radar_input)
ax[0, 2].set_title('Radar Input')

ax[1, 0].imshow(vis_optical_input)
ax[1, 0].set_title('Optical GT')
ax[2, 0].imshow(vis_radar_input)
ax[2, 0].set_title('Radar GT')

ax[1, 3].imshow(vis_optical_mask)
# put the title to the right
ax[1, 3].set_title('Optical Mask')
ax[2, 3].imshow(vis_radar_mask)
ax[2, 3].set_title('Radar Mask')

if vis_optical_recon_o is not None:
    ax[1, 1].imshow(vis_optical_recon_o)
if vis_optical_recon_r is not None:
    ax[2, 1].imshow(vis_optical_recon_r)

if vis_radar_recon_o is not None:
    ax[1, 2].imshow(vis_radar_recon_o)
if vis_radar_recon_r is not None:
    ax[2, 2].imshow(vis_radar_recon_r)

# close all the ticks
for axi in ax.ravel():
    axi.axis('off')

fig.suptitle(exp_name, fontsize=32)

plt.tight_layout()

In [ ]:
mpl.rcParams['axes.titlesize'] = 32

gt_all = np.concatenate([optical_input, radar_input], axis=2)
# for gt_all, optical_recon_all, radar_recon_all, pop channel 10
gt_all_ = np.delete(gt_all, 10, axis=2)
optical_recon_all_ = np.delete(optical_recon_all, 10, axis=2)
radar_recon_all_ = np.delete(radar_recon_all, 10, axis=2)
mm_recon_all_ = np.delete(mm_recon_all, 10, axis=2)
optical_channel_mask_ = np.delete(optical_channel_mask, 10, axis=0) if optical_channel_mask is not None else None
mm_channel_mask_ = np.delete(mm_channel_mask, 10, axis=0) if mm_channel_mask is not None else None
n_channels = gt_all_.shape[2]
# plot every channel optical_recon_all in a 3x5 subplot
# plt.rcParams.update({'font.size': 8})
# fig, ax = plt.subplots(3, n_channels, figsize=(17, 4))

fig = plt.figure(figsize=(56, 16)) 
gs = GridSpec(4, n_channels+1, figure=fig)

for j in range(n_channels):
    ax = fig.add_subplot(gs[0, j+1])
    ax.imshow(gt_all_[:, :, j])
    # ax[0, j].imshow(gt_all_[:, :, j])
    if j > 11:
        # ax[0, j].set_title(f'RC {j-12}')
        ax.set_title(f'RC {j-12}')
    elif optical_channel_mask_ is not None and j < len(optical_channel_mask_):
        if optical_channel_mask_[j] == 0:
            # ax[0, j].set_title(f'OC {j}')
            ax.set_title(f'OC {j}')
        else:
            # ax[0, j].set_title(f'OC {j} (m)')
            ax.set_title(f'OC {j} (m)')
    else:
        # ax[0, j].set_title(f'OC {j}')
        ax.set_title(f'OC {j}')
    # ax[0, j].axis('off')
    ax.axis('off')
for j in range(n_channels):
    ax = fig.add_subplot(gs[1, j+1])
    ax.imshow(optical_recon_all_[:, :, j])
    # ax[1, j].imshow(optical_recon_all_[:, :, j])
    # ax[1, j].set_title(f'Channel {j}')
    # ax[1, j].axis('off')
    ax.axis('off')
for j in range(n_channels):
    ax = fig.add_subplot(gs[2, j+1])
    # ax[2, j].imshow(radar_recon_all_[:, :, j])
    ax.imshow(radar_recon_all_[:, :, j])
    # ax[1, j].set_title(f'Channel {j}')
    # ax[2, j].axis('off')
    ax.axis('off')
for j in range(n_channels):
    ax = fig.add_subplot(gs[3, j+1])
    # ax[2, j].imshow(radar_recon_all_[:, :, j])
    ax.imshow(mm_recon_all_[:, :, j])
    # ax[1, j].set_title(f'Channel {j}')
    # ax[2, j].axis('off')
    ax.axis('off')
fig.suptitle(exp_name, fontsize=38)
row_labels = ['Ground Truth', 'MSI Recon', 'SAR Recon', 'MM Recon']  # Adjust as needed
# Set the x coordinate for the labels, this will need to be adjusted based on your layout
x_coord = 1.1

# Add the labels to the left of the first column of images
for i, label in enumerate(row_labels):
    ax = fig.add_subplot(gs[i, 0])
    ax.axis('off')  # Turn off the axis if not needed

    # Place the text relative to the current axis
    ax.text(x_coord, 0.5, label, va='center', ha='right', transform=ax.transAxes, fontsize=32)

plt.show()

## Feature Visualization

In [ ]:
from sklearn.decomposition import PCA
import cv2

In [ ]:
data = get_data(cfg)
# samples = next(iter(data['train'].dataloader))
# for key in samples:
    # samples[key] = samples[key].to(device)

# n_batchs = 2
# # acumulate 256 samples
# samples = {}
# for i, batch in enumerate(data['train'].dataloader):
#     for key in batch:
#         if key not in samples:
#             samples[key] = []
#         samples[key].append(batch[key].to(device))
#     if i == n_batchs:
#         break

# for key in samples:
#     samples[key] = torch.cat(samples[key], dim=0)

In [ ]:
# samples['image'].shape

In [ ]:
# samples['radar'].std(), samples['radar'].mean(), samples['image'].std(), samples['image'].mean()

In [ ]:
model.eval()
optical_patch = []
radar_patch = []
mm_patch = []
sample_images = []
for i, samples in enumerate(tqdm(data['train'].dataloader)):
    for key in samples:
        samples[key] = samples[key].to(device)
    with torch.no_grad():
        encode_optical = model.encoder.forward(samples['image'], None, return_dict=True)
        encode_radar = model.encoder.forward(None, samples['radar'], return_dict=True)
        encode_mm = model.encoder.forward(samples['image'], samples['radar'], return_dict=True)
    optical_patch.append(encode_optical['patch_tokens'].cpu().numpy())
    radar_patch.append(encode_radar['patch_tokens'].cpu().numpy())
    mm_patch.append(encode_mm['patch_tokens'].cpu().numpy())    
    sample_images.append(samples['image'].cpu().numpy())
    if i >= 4:
        break

optical_patch = np.concatenate(optical_patch, axis=0)
radar_patch = np.concatenate(radar_patch, axis=0)
mm_patch = np.concatenate(mm_patch, axis=0)
sample_images = np.concatenate(sample_images, axis=0)

In [ ]:
# optical_patch = encode_optical['patch_tokens']
# radar_patch = encode_radar['patch_tokens']
# mm_patch = encode_mm['patch_tokens']
optical_patch.shape, radar_patch.shape, mm_patch.shape, sample_images.shape

In [ ]:
def pca_vis_patch(patch, n_components=3):
    # perform PCA on patch
    pca = PCA(n_components=n_components)
    B, L, D = patch.shape
    patch_ = patch.reshape(B*L, D)
    try:
        patch_ = patch_.cpu().numpy()
    except:
        pass
    pca.fit(patch_)
    patch_pca = pca.transform(patch_)

    background_mask = patch_pca[:, 0] > 128
    foreground_mask = ~background_mask
    foregroud_patchs = patch_[foreground_mask]

    # do the second PCA on the foreground patchs
    pca_fore = PCA(n_components=n_components)
    pca_fore.fit(foregroud_patchs)
    patch_pca_fore = pca_fore.transform(foregroud_patchs)

    processed_patches = np.zeros_like(patch_pca)
    processed_patches[foreground_mask] = patch_pca_fore
    preprocessed_patches = patch_pca.reshape(B, int(L**0.5), int(L**0.5), 3)
    processed_patches = processed_patches.reshape(B, int(L**0.5), int(L**0.5), 3)

    # preprocessed_patches = norm_image(preprocessed_patches)
    # processed_patches = norm_image(processed_patches)
    return preprocessed_patches, processed_patches

In [ ]:
optical_pre_patch, _ = pca_vis_patch(optical_patch)
radar_pre_patch, _ = pca_vis_patch(radar_patch)
mm_pre_patch, _ = pca_vis_patch(mm_patch)

In [ ]:
mpl.rcParams['axes.titlesize'] = 20

fig, ax = plt.subplots(4, 7, figsize=(36, 22))

sample_idices = [28, 138, 309, 303]
# sample_idices = np.random.choice(len(sample_images), 4, replace=False)
print(sample_idices)

for i, sample_idx in enumerate(sample_idices):
    sample_img = sample_images[sample_idx].transpose(1, 2, 0)[:, :, [3, 2, 1]]
    sample_img = norm_image(sample_img)

    # pca_img = processed_patches[sample_idx]
    pca_img = optical_pre_patch[sample_idx]
    # resize the image to 224x224
    pca_img = cv2.resize(pca_img, sample_img.shape[:2], interpolation=cv2.INTER_CUBIC)
    # pca_img = np.repeat(np.repeat(pca_img, 16, axis=0), 16, axis=1)
    pca_img = norm_image(pca_img)

    pca_radar = radar_pre_patch[sample_idx]
    pca_radar = cv2.resize(pca_radar, sample_img.shape[:2], interpolation=cv2.INTER_CUBIC)
    pca_radar = norm_image(pca_radar)

    pca_mm = mm_pre_patch[sample_idx]
    pca_mm = cv2.resize(pca_mm, sample_img.shape[:2], interpolation=cv2.INTER_CUBIC)
    pca_mm = norm_image(pca_mm)

    alpha = 0.5
    beta = (1.0 - alpha)
    gamma = 0
    overlaid_img = cv2.addWeighted(sample_img, alpha, pca_img, beta, gamma)
    overlaid_radar = cv2.addWeighted(sample_img, alpha, pca_radar, beta, gamma)
    overlaid_mm = cv2.addWeighted(sample_img, alpha, pca_mm, beta, gamma)

    ax[i, 0].imshow(sample_img)
    ax[i, 1].imshow(pca_img)
    ax[i, 2].imshow(overlaid_img)
    ax[i, 3].imshow(pca_radar)
    ax[i, 4].imshow(overlaid_radar)
    ax[i, 5].imshow(pca_mm)
    ax[i, 6].imshow(overlaid_mm)
    

    ax[i, 0].set_axis_off()
    ax[i, 1].set_axis_off()
    ax[i, 2].set_axis_off()
    ax[i, 3].set_axis_off()
    ax[i, 4].set_axis_off()
    ax[i, 5].set_axis_off()
    ax[i, 6].set_axis_off()

ax[0, 0].set_title('Original Image')
ax[0, 1].set_title('Optical PCA Patches')
ax[0, 2].set_title('Optical Overlay Image')
ax[0, 3].set_title('Radar PCA Patches')
ax[0, 4].set_title('Radar Overlay Image')
ax[0, 5].set_title('MM PCA Patches')
ax[0, 6].set_title('MM Overlay Image')